references <br>
http://blog.theeluwin.kr/post/147587579528/python-crfsuite%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%B4%EC%84%9C-%ED%95%9C%EA%B5%AD%EC%96%B4-%EC%9E%90%EB%8F%99-%EB%9D%84%EC%96%B4%EC%93%B0%EA%B8%B0%EB%A5%BC-%ED%95%99%EC%8A%B5%ED%95%B4%EB%B3%B4%EC%9E%90

In [4]:
import os
import glob
import mxnet as mx
from mxnet.gluon import nn, rnn
import gluonnlp as nlp
import pandas as pd
import numpy as np
from mxnet import gluon, autograd
from mxnet import nd
import mxnet as mx
import time
import itertools
from tqdm import tqdm
import multiprocessing as mp
from konlpy.tag import Twitter
twitter = Twitter()

## read all review files

In [17]:
path = "101 movie/reviews_2012"
allFiles = glob.glob(path + "/*.csv")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, sep='\t')
    list_.append(df)
reviews = pd.concat(list_, axis = 0, ignore_index = True)

In [18]:
reviews[:10]

,Unnamed: 0,comments,reg_date,source,pn,topic,mid
0,0,BEST속옷만 입은 채 피범벅이 된 김민희 표정 소름 쫙,2013.07.05,n,1,1,20124050
1,1,BEST살인은 부당했지만 .. 여자의 인생은 참 슬프네요 기구하고 ... 남자를위해...,2013.06.20,n,1,0,20124050
2,2,BEST이선균은 대사가 잘 안들릴정도로 가끔 너무 작게 말한다.,2013.06.16,n,1,1,20124050
3,3,BEST간만에 정말 빠져들고 스릴러 다운 영화였다.추격자 이후 이렇게 제대로된 스릴...,2013.06.28,n,1,1,20124050
4,4,BEST마음이 아픈 스릴러.. 화차에 몸을 싣기로 한 그녀의 선택이 너무 아프다,2013.06.22,n,1,3,20124050
5,5,기대않고봤는데 몰입도가 대단했음. 김민희의 연기가 이 정도일줄이야 보는내내 감탄하고...,2013.06.01,n,1,1,20124050
6,6,그렇게 해서라도 행복해지고 싶었던 불쌍한 인물.. 잘 완성된 작품에 비해 흥행이 아...,2013.06.15,n,1,0,20124050
7,7,국산영화중에 진짜 잘만든영화네요.. 연기들 잘하네,2013.06.12,n,1,1,20124050
8,8,누구라도 강선영의 상황에 처해진다면 강선영을 비난하지만은 못할것같다.,2013.06.23,n,1,0,20124050
9,9,이 정도면 진짜 잘만든거 와우..,2013.06.08,n,1,0,20124050


## save all files to one

In [19]:
# all files
reviews.comments.to_csv("all_reviews.txt", header=None, index=None, sep=" ", mode="a")

# samples
reviews.comments[:100].to_csv("samples.txt", header=None, index=None, sep=" ", mode="a")

comments = reviews.comments[:100]
comments[:10]

0                    BEST속옷만 입은 채 피범벅이 된 김민희 표정 소름 쫙  
1    BEST살인은 부당했지만 .. 여자의 인생은 참 슬프네요 기구하고 ... 남자를위해...
2                BEST이선균은 대사가 잘 안들릴정도로 가끔 너무 작게 말한다.  
3    BEST간만에 정말 빠져들고 스릴러 다운 영화였다.추격자 이후 이렇게 제대로된 스릴...
4       BEST마음이 아픈 스릴러.. 화차에 몸을 싣기로 한 그녀의 선택이 너무 아프다  
5    기대않고봤는데 몰입도가 대단했음. 김민희의 연기가 이 정도일줄이야 보는내내 감탄하고...
6    그렇게 해서라도 행복해지고 싶었던 불쌍한 인물.. 잘 완성된 작품에 비해 흥행이 아...
7                        국산영화중에 진짜 잘만든영화네요.. 연기들 잘하네  
8             누구라도 강선영의 상황에 처해진다면 강선영을 비난하지만은 못할것같다.  
9                                 이 정도면 진짜 잘만든거 와우..  
Name: comments, dtype: object

## get all actors

In [78]:
all_actors = pd.read_csv("101 movie/all_actors.csv",index_col=None, header=None, sep=',')
all_actors[:10]

,0,1
0,0,조쉬 더하멜
1,1,존 말코비치
2,2,로지 헌팅턴-휘틀리
3,3,샤이아 라보프
4,4,패트릭 뎀시
5,5,톰 크루즈
6,6,제레미 레너
7,7,박해일
8,8,류승룡
9,9,김무열


## get all directors

In [30]:
all_directs = pd.read_csv("101 movie/all_directors.csv",index_col=None, header=None, sep=',')
all_directs[:10]

,0,1
0,344,마이클 베이
1,345,브래드 버드
2,346,김한민
3,347,강형철
4,348,이한
5,349,여인영
6,350,김석윤
7,351,황동혁
8,352,데이빗 예이츠
9,353,숀 레비


In [149]:
def raw2corpus_(raw_path, train_path, tag, corpus_path):
    raw = codecs.open(raw_path, encoding='utf-8')
    raw_sentences = raw.read().split('\n')
    corpus = codecs.open(corpus_path, 'w', encoding='utf-8')
    train_data = pd.read_csv(train_path,index_col=None, header=None, sep=',')
    sentences = []
    for raw_sentence in raw_sentences:
        if not raw_sentence:
            continue
        text = re.sub(r'(\ )+', ' ', raw_sentence).strip()
        text = text.replace('"', '')
        word_list = twitter.pos(text)
        taggeds = []
        for i in range(len(word_list)):
            word = word_list[i]
            if (word[1] == "Noun") and (len(word[0]) > 1):
                hit = all_actors[train_data[1].str.match(word[0])]
                if len(hit):
                    taggeds.append('{}/' + tag.format(word[0]))
                else:
                    taggeds.append('{}/O'.format(word[0]))
            else:
                taggeds.append('{}/O'.format(word[0]))
        sentences.append(' '.join(taggeds))
    corpus.write('\n'.join(sentences))
    return sentences

In [152]:
# all review data // all actor data // file for save
raw2corpus_("samples.txt", "101 movie/all_actors.csv", "ACTOR", "actor_list.txt")
raw2corpus_("samples.txt", "101 movie/all_directors.csv", "DIRECTOR", "director_list.txt")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [50]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function

import re
import codecs

def raw2corpus(raw_path, corpus_path):
    raw = codecs.open(raw_path, encoding='utf-8')
    raw_sentences = raw.read().split('\n')
    corpus = codecs.open(corpus_path, 'w', encoding='utf-8')
    sentences = []
    for raw_sentence in raw_sentences:
        if not raw_sentence:
            continue
        text = re.sub(r'(\ )+', ' ', raw_sentence).strip()
        text = text.replace('"', '')
        taggeds = []
        for i in range(len(text)):
            if i == 0:
                taggeds.append('{}/B'.format(text[i]))
            elif text[i] != ' ':
                successor = text[i - 1]
                if successor == ' ':
                    taggeds.append('{}/B'.format(text[i]))
                else:
                    taggeds.append('{}/I'.format(text[i]))
        sentences.append(' '.join(taggeds))
    corpus.write('\n'.join(sentences))
    return sentences

In [51]:
raw2corpus("samples.txt", "train.txt")

['B/B E/I S/I T/I 속/I 옷/I 만/I 입/B 은/I 채/B 피/B 범/I 벅/I 이/I 된/B 김/B 민/I 희/I 표/B 정/I 소/B 름/I 쫙/B',
 'B/B E/I S/I T/I 살/I 인/I 은/I 부/B 당/I 했/I 지/I 만/I ./B ./I 여/B 자/I 의/I 인/B 생/I 은/I 참/B 슬/B 프/I 네/I 요/I 기/B 구/I 하/I 고/I ./B ./I ./I 남/B 자/I 를/I 위/I 해/I 마/B 지/I 막/I 에/I 사/B 랑/I 을/I 하/B 지/I 않/I 았/I 다/I 고/I 한/B 거/I 같/I 아/I 요/I ./B ./I ./I 사/B 실/I 은/I 누/B 구/I 보/I 다/I 행/B 복/I 해/I 지/I 고/I 싶/B 으/I 면/I 서/I ./B ./I ./I',
 'B/B E/I S/I T/I 이/I 선/I 균/I 은/I 대/B 사/I 가/I 잘/B 안/B 들/I 릴/I 정/I 도/I 로/I 가/B 끔/I 너/B 무/I 작/B 게/I 말/B 한/I 다/I ./I',
 'B/B E/I S/I T/I 간/I 만/I 에/I 정/B 말/I 빠/B 져/I 들/I 고/I 스/B 릴/I 러/I 다/B 운/I 영/B 화/I 였/I 다/I ./I 추/I 격/I 자/I 이/B 후/I 이/B 렇/I 게/I 제/B 대/I 로/I 된/I 스/B 릴/I 러/I 를/I 보/B 다/I 니/I !/I 평/I 점/I 들/I 이/I 너/B 무/I 박/B 하/I 다/I ./I 그/I 리/I 고/I 김/B 민/I 희/I 가/I 이/B 렇/I 게/I 연/B 기/I 를/I 잘/B 했/B 었/I 나/I ./I 캐/I 릭/I 터/I 에/I 너/B 무/I 나/I 도/I 잘/B 소/B 화/I 했/I 다/I ./I',
 'B/B E/I S/I T/I 마/I 음/I 이/I 아/B 픈/I 스/B 릴/I 러/I ./I ./I 화/B 차/I 에/I 몸/B 을/I 싣/B 기/I 로/I 한/B 그/B 녀/I 의/I 선/B 택/I 이/I 너/B 무/I 아/B 프/

In [52]:
def corpus2sent(path):
    corpus = codecs.open(path, encoding='utf-8').read()
    raws = corpus.split('\n')
    sentences = []
    for raw in raws:
        tokens = raw.split(' ')
        sentence = []
        for token in tokens:
            try:
                word, tag = token.split('/')
                if word and tag:
                    sentence.append([word, tag])
            except:
                pass
        sentences.append(sentence)
    return sentences

In [53]:
sample = corpus2sent('train.txt')[7]

In [54]:
sample

[['국', 'B'],
 ['산', 'I'],
 ['영', 'I'],
 ['화', 'I'],
 ['중', 'I'],
 ['에', 'I'],
 ['진', 'B'],
 ['짜', 'I'],
 ['잘', 'B'],
 ['만', 'I'],
 ['든', 'I'],
 ['영', 'I'],
 ['화', 'I'],
 ['네', 'I'],
 ['요', 'I'],
 ['.', 'I'],
 ['.', 'I'],
 ['연', 'B'],
 ['기', 'I'],
 ['들', 'I'],
 ['잘', 'B'],
 ['하', 'I'],
 ['네', 'I']]

In [55]:
def index2feature(sent, i, offset):
    word, tag = sent[i + offset]
    if offset < 0:
        sign = ''
    else:
        sign = '+'
    return '{}{}:word={}'.format(sign, offset, word)

def word2features(sent, i):
    L = len(sent)
    word, tag = sent[i]
    features = ['bias']
    features.append(index2feature(sent, i, 0))
    if i > 1:
        features.append(index2feature(sent, i, -2))
    if i > 0:
        features.append(index2feature(sent, i, -1))
    else:
        features.append('bos')
    if i < L - 2:
        features.append(index2feature(sent, i, 2))
    if i < L - 1:
        features.append(index2feature(sent, i, 1))
    else:
        features.append('eos')
    return features

def sent2words(sent):
    return [word for word, tag in sent]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [56]:
raw2corpus("samples.txt", 'test.txt')

['B/B E/I S/I T/I 속/I 옷/I 만/I 입/B 은/I 채/B 피/B 범/I 벅/I 이/I 된/B 김/B 민/I 희/I 표/B 정/I 소/B 름/I 쫙/B',
 'B/B E/I S/I T/I 살/I 인/I 은/I 부/B 당/I 했/I 지/I 만/I ./B ./I 여/B 자/I 의/I 인/B 생/I 은/I 참/B 슬/B 프/I 네/I 요/I 기/B 구/I 하/I 고/I ./B ./I ./I 남/B 자/I 를/I 위/I 해/I 마/B 지/I 막/I 에/I 사/B 랑/I 을/I 하/B 지/I 않/I 았/I 다/I 고/I 한/B 거/I 같/I 아/I 요/I ./B ./I ./I 사/B 실/I 은/I 누/B 구/I 보/I 다/I 행/B 복/I 해/I 지/I 고/I 싶/B 으/I 면/I 서/I ./B ./I ./I',
 'B/B E/I S/I T/I 이/I 선/I 균/I 은/I 대/B 사/I 가/I 잘/B 안/B 들/I 릴/I 정/I 도/I 로/I 가/B 끔/I 너/B 무/I 작/B 게/I 말/B 한/I 다/I ./I',
 'B/B E/I S/I T/I 간/I 만/I 에/I 정/B 말/I 빠/B 져/I 들/I 고/I 스/B 릴/I 러/I 다/B 운/I 영/B 화/I 였/I 다/I ./I 추/I 격/I 자/I 이/B 후/I 이/B 렇/I 게/I 제/B 대/I 로/I 된/I 스/B 릴/I 러/I 를/I 보/B 다/I 니/I !/I 평/I 점/I 들/I 이/I 너/B 무/I 박/B 하/I 다/I ./I 그/I 리/I 고/I 김/B 민/I 희/I 가/I 이/B 렇/I 게/I 연/B 기/I 를/I 잘/B 했/B 었/I 나/I ./I 캐/I 릭/I 터/I 에/I 너/B 무/I 나/I 도/I 잘/B 소/B 화/I 했/I 다/I ./I',
 'B/B E/I S/I T/I 마/I 음/I 이/I 아/B 픈/I 스/B 릴/I 러/I ./I ./I 화/B 차/I 에/I 몸/B 을/I 싣/B 기/I 로/I 한/B 그/B 녀/I 의/I 선/B 택/I 이/I 너/B 무/I 아/B 프/

In [57]:
import pycrfsuite

train_sents = corpus2sent('train.txt')
test_sents = corpus2sent('test.txt')
train_x = [sent2features(sent) for sent in train_sents]
train_y = [sent2tags(sent) for sent in train_sents]
test_x = [sent2features(sent) for sent in test_sents]
test_y = [sent2tags(sent) for sent in test_sents]

ModuleNotFoundError: No module named 'pycrfsuite'

In [58]:
train_sents

NameError: name 'train_sents' is not defined

In [ ]:
test_sents

In [59]:
train_x

NameError: name 'train_x' is not defined

In [60]:
train_y

NameError: name 'train_y' is not defined

In [61]:
test_sents

NameError: name 'test_sents' is not defined

In [171]:
test_x

[['B',
  'E',
  'S',
  'T',
  '가',
  '슴',
  '세',
  '개',
  '녀',
  '가',
  '너',
  '무',
  '인',
  '상',
  '깊',
  '었',
  '다'],
 ['B',
  'E',
  'S',
  'T',
  '결',
  '국',
  '리',
  '콜',
  '사',
  '에',
  '서',
  '환',
  '상',
  '을',
  '끝',
  '내',
  '주',
  '게',
  '경',
  '험',
  '했',
  '다',
  '는',
  '얘',
  '기',
  '같',
  '네',
  '요',
  ',',
  '리',
  '메',
  '이',
  '크',
  '수',
  '준',
  '도',
  '괜',
  '찮',
  '았',
  '음',
  '.'],
 ['B',
  'E',
  'S',
  'T',
  '결',
  '국',
  '가',
  '상',
  '이',
  '였',
  '을',
  '까',
  '현',
  '실',
  '이',
  '였',
  '을',
  '까',
  '.',
  '.',
  '많',
  '은',
  '생',
  '각',
  '을',
  '하',
  '게',
  '만',
  '든',
  '영',
  '화'],
 ['B',
  'E',
  'S',
  'T',
  '원',
  '작',
  '의',
  '색',
  '안',
  '경',
  '을',
  '빼',
  '고',
  '보',
  '면',
  '참',
  '잘',
  '만',
  '든',
  '영',
  '화',
  ',',
  '원',
  '작',
  '팬',
  '들',
  '을',
  '위',
  '한',
  '오',
  '마',
  '쥬',
  '에',
  '도',
  '충',
  '실',
  '함'],
 ['B',
  'E',
  'S',
  'T',
  '볼',
  '만',
  '하',
  '네',
  '요',
  '.',
  '.',
  '.',
  '적',
  '이',
  '좀',
  '허',
  

In [172]:
trainer = pycrfsuite.Trainer()

In [173]:
for x, y in zip(train_x, train_y):
    trainer.append(x, y)

In [174]:
trainer.train('space.crfsuite')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 3699
Seconds required: 0.014

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 2147483647
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 5263.078372
Feature norm: 1.000000
Error norm: 3311.635310
Active features: 3323
Line search trials: 1
Line search step: 0.000269
Seconds required for this iteration: 0.004

***** Iteration #2 *****
Loss: 4479.313954
Feature norm: 0.731168
Error norm: 1101.128967
Active features: 3699
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 4336.408128
Feature norm: 0.764355
Error norm: 612.080783
Active features: 3699
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

In [175]:
tagger = pycrfsuite.Tagger()
tagger.open('space.crfsuite')

In [180]:
sent = test_sents[0]
sent

[['B', 'B'],
 ['E', 'I'],
 ['S', 'I'],
 ['T', 'I'],
 ['가', 'I'],
 ['슴', 'I'],
 ['세', 'I'],
 ['개', 'I'],
 ['녀', 'I'],
 ['가', 'I'],
 ['너', 'B'],
 ['무', 'I'],
 ['인', 'B'],
 ['상', 'I'],
 ['깊', 'I'],
 ['었', 'I'],
 ['다', 'I']]

In [181]:
print("Sentence: ", ' '.join(sent2words(sent)))

Sentence:  B E S T 가 슴 세 개 녀 가 너 무 인 상 깊 었 다


In [182]:
print("Correct:  ", ' '.join(sent2tags(sent)))

Correct:   B I I I I I I I I I B I B I I I I


In [183]:
print("Predicted:", ' '.join(tagger.tag(sent2features(sent))))

Predicted: B I I I I I I I I I B I B I I I I
